In [52]:
import numpy as np
import functools as f
import operator as op

In [53]:
X = [0.87267, 1.22173, 1.57080, 1.91986, 2.26893, 2.61799, 2.9670]
Y = [0.00082, 0.01039, 0.07037, 0.32762, 1.18669, 3.59003, 9.4835]

In [54]:
def L(i):
    x_i = X[i]
    nom_list = [np.poly1d([1, -x_k]) for x_k in X if x_k != x_i]
    nom = f.reduce(op.mul, nom_list)
    denom = np.prod([x_i - x_k for x_k in X if x_k != x_i])
    return nom/denom

In [55]:
Ls = [L(i) for i in range(len(X))]
P = sum([l*y for l, y in zip(Ls, Y)])
print ('\nКоэффициенты интерполяционного многочлена:')
print ('  '.join('{:.4E}'.format(c) for c in P.coef))


Коэффициенты интерполяционного многочлена:
1.4306E-01  -1.0474E+00  3.5672E+00  -6.7628E+00  7.3184E+00  -4.2198E+00  1.0025E+00


In [56]:
def a(i):
    x = X[i]
    x_1 = X[i+1]
    y = P(x)
    y_1 = P(x_1)
    dx = x_1 - x
    dy = y_1 - y
    P_1 = np.polyder(P)
    a_0 = -P_1(x_1) * x**2 * x_1 * dx + y_1 * x**2 * (3*x_1 - x) + y * x_1**2 * (x_1 - 3*x) - P_1(x) * x * x_1**2 * dx
    a_1 = P_1(x_1) * x * (2*x_1 + x) * dx - 6*dy * x * x_1 + P_1(x) * x_1 * (x_1 + 2*x) * dx
    a_2 = -P_1(x_1) * dx * (x_1 + 2*x) + 3*dy * (x_1 + x) - P_1(x) * dx * (x + 2*x_1)
    a_3 = P_1(x_1) * dx - 2*dy + P_1(x) * dx
    return [a / dx**3 for a in [a_0, a_1, a_2, a_3]]

In [57]:
def S_i(i, x):
    return np.poly1d(a(i)[::-1])(x)

In [58]:
def S(x):
    if x < X[0] or x > X[-1]:
        print('Error: Значение не входит в заданный диапазон')
        return 0
    for i in range(6):
            if X[i] <= x <= X[i+1]:
                return S_i(i, x)

In [ ]:
A = [a(i) for i in range(6)]
print('\nТаблица значений коэффициентов сплайнов:')
print('a_0', 'a_1', 'a_2', 'a_3', sep = '          ')
print('\n'.join(['  '.join(['{:.4E}'.format(item) for item in row]) for row in A]))
print()
while True:
    x = float(input('x ='))
    print('Значение сплайна в заданной точке: {:.4E}'.format(S(x)))
    print('Значение интерполяционного многочлена в заданной точке: {:.4E}'.format(P(x)))
    


Таблица значений коэффициентов сплайнов:
a_0          a_1          a_2          a_3
1.4146E-01  -3.2749E-01  2.1647E-01  -2.9656E-02
-8.9528E-01  2.2180E+00  -1.8668E+00  5.3868E-01
-4.6519E+00  9.3121E+00  -6.3319E+00  1.4753E+00
-1.9430E+01  3.2208E+01  -1.8156E+01  3.5105E+00
-6.5350E+01  9.2576E+01  -4.4609E+01  7.3743E+00
-1.8198E+02  2.2569E+02  -9.5251E+01  1.3796E+01

